# import

In [16]:
from Database.Database import DataFunFilm , DataFunDist
import pandas as pd
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader as api
import gensim
import pyemd
from Handlers.preProc import PrepText
import re
import numpy as np
import zipfile
import wget
from tqdm import tqdm
import pickle

# Загрузка базы данных

In [9]:
db = DataFunFilm()
films = db.getTableDF()
films

,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits,prepare_description
0,1,1043658,tt4202274,Шоу Грязного Фрэнка,The Filthy Frank Show,https://kinopoiskapiunofficial.tech/images/pos...,9.7,32929,NaN,0,2011.0,12.0,фэнтези комедия музыка короткометражка,https://www.kinopoisk.ru/series/1043658/,None,Шоу повествует о героической борьбе сэра Фрэнс...,9.3,None,None,шоу_PROPN повествовать_VERB о_ADP героический...
1,2,1309325,None,Космос,,https://kinopoiskapiunofficial.tech/images/pos...,NaN,0,NaN,0,2019.0,NaN,документальный,https://www.kinopoisk.ru/series/1309325/,None,Астрофизик Нил Деграсс Тайсон встает за штурва...,9.3,Нил Деграсс Тайсон отправляется в мысленное пу...,age18,астрофизик_NOUN нил_VERB деграсс::тайсон_PROPN...
2,3,77044,tt0108778,Друзья,Friends,https://kinopoiskapiunofficial.tech/images/pos...,9.0,937875,NaN,0,1994.0,22.0,мелодрама комедия,https://www.kinopoisk.ru/series/77044/,Everybody needs Friends!,"Шестеро друзей – Рейчел, Моника, Фиби, Джоуи, ...",9.2,"Шестеро друзей, две квартиры и много юмора. Си...",age16,шестеро_NUM друг_NOUN рейчел_PROPN моника_PRO...
3,4,972889,tt4926348,BadComedian,None,https://kinopoiskapiunofficial.tech/images/pos...,7.7,793,NaN,0,2011.0,NaN,комедия,https://www.kinopoisk.ru/series/972889/,None,"Юмористическое шоу, высмеивающее плохое кино. ...",9.2,None,age18,юмористический_ADJ шоу_NOUN высмеивать_VERB пл...
4,5,674243,None,Счастливые люди,None,https://kinopoiskapiunofficial.tech/images/pos...,8.8,120,NaN,0,2008.0,55.0,документальный,https://www.kinopoisk.ru/series/674243/,None,Документальный фильм о трудах и днях охотников...,9.2,None,None,документальный_ADJ фильм_NOUN о_ADP труд_NOUN ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66752,66753,83307,tt0041640,Мари из порта,La Marie du port,https://kinopoiskapiunofficial.tech/images/pos...,6.7,395,NaN,0,1950.0,88.0,драма мелодрама,https://www.kinopoisk.ru/film/83307/,None,"Анри Шателяру уже за сорок, он владеет рестора...",7.0,None,None,анри_PROPN шателяр_PROPN уже_ADV за_ADP сорок...
66753,66754,219369,tt0071859,Путешественник,Mossafer,https://kinopoiskapiunofficial.tech/images/pos...,7.5,1695,NaN,0,1974.0,83.0,драма,https://www.kinopoisk.ru/film/219369/,None,Парнишке-школьнику по имени Кассем из маленько...,7.0,None,None,парнишка_NOUN школьник_NOUN по_ADP имя_NOUN ка...
66754,66755,47614,tt0211372,Сын француза,Le fils du Français,https://kinopoiskapiunofficial.tech/images/pos...,5.0,282,NaN,0,1999.0,107.0,приключения комедия семейный,https://www.kinopoisk.ru/film/47614/,None,У Бенжамена 9 лет нет больше мамы. Он хочет на...,7.0,None,None,в_ADP бенжамен_PROPN 9_NUM год_NOUN нет_VERB ...
66755,66756,977755,tt9378362,Звоните ДиКаприо!,None,https://kinopoiskapiunofficial.tech/images/pos...,6.7,496,NaN,0,2018.0,48.0,драма комедия,https://www.kinopoisk.ru/series/977755/,Что останется после тебя?,Егор Румянцев – звезда популярного сериала про...,7.0,None,age18,егор::румянцев_PROPN звезда_NOUN популярный_A...


 # Подготовка описаний

In [114]:
prep = PrepText()


Loading the model...


In [222]:
prepDisc = []

for i in tqdm(films["description"]):
    prepDisc.append(prep.tag_ud(i).split())
films["prepDisc"] = prepDisc

100%|██████████| 66757/66757 [45:56<00:00, 24.22it/s] 


# Загрузка модели

In [7]:
model_url = 'http://vectors.nlpl.eu/repository/20/220.zip'
# m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

# Поиск похожих фильмов

In [26]:
def moreSimilar(discripts, id):
    disc = discripts[id]
    dist_lst = []
    for i in tqdm(range(len(discripts))):
        if i != id:
            dist = model.wmdistance(discripts[i], disc)
            dist_lst.append([dist,i])
    dist_lst.sort(key = lambda x: x[0])
    return dist_lst

In [27]:
def getSimFilm(num, col = 5):
    print("Фильм :")
    print(films.iloc[num])
    print("Самый схожий:")
    lst = moreSimilar(list(films["prepDisc"]),num)
    for i in range(col):
        print(films.iloc[lst[i][1]])

In [28]:
getSimFilm(52822)

Фильм :
id                                                                       52823
kinopoisk_id                                                           1234853
imdb_id                                                             tt14694956
name_ru                                                          Мистер Нокаут
name_original                                                             None
poster_url                   https://kinopoiskapiunofficial.tech/images/pos...
rating_imdb                                                                NaN
rating_imdb_vote_count                                                       0
rating_film_critics                                                        NaN
rating_critics_vote_count                                                    0
year                                                                    2022.0
film_length                                                              117.0
genres                                      

100%|██████████| 66757/66757 [05:00<00:00, 222.49it/s]

id                                                                        9666
kinopoisk_id                                                            840817
imdb_id                                                              tt5987402
name_ru                                                         Движение вверх
name_original                                                             None
poster_url                   https://kinopoiskapiunofficial.tech/images/pos...
rating_imdb                                                                6.7
rating_imdb_vote_count                                                    4826
rating_film_critics                                                        NaN
rating_critics_vote_count                                                    0
year                                                                    2017.0
film_length                                                              133.0
genres                                              

# Добавление расстояний в базу данных

In [282]:
films.to_pickle("films.pkl")

In [74]:
done_ids = []
with open('done_ids.pkl', 'wb') as f:
    pickle.dump(done_ids, f)

In [3]:
films = pd.read_pickle("films.pkl")

In [6]:
db_dist = DataFunDist()

In [ ]:
def calculate_distance(num):
    lst = moreSimilar(list(films["prepDisc"]),num)
    for i in lst:
        db_dist.addDistance(films.iloc[num]["id"], films.iloc[i[1]]["id"], i[0])

In [ ]:
films.iloc[5]["id"]

In [ ]:
def moreSimilar(films, id, prepDisc):
    dist_lst = []
    for i in tqdm(films):
        if i[1] != id:
            dist = model.wmdistance(i[0], prepDisc)
            dist_lst.append([dist,i[1]])
    dist_lst.sort(key = lambda x: x[0])
    return dist_lst

In [ ]:
def calculate_distance(films, id, prepDisc):
    return moreSimilar(list(zip(list(films["prepDisc"]),list(films["id"]))), id, prepDisc)

In [ ]:
def add_to_DB(num,db):
    lst = calculate_distance(films, films.iloc[num]["id"], films.iloc[num]["prepDisc"])
    for i in lst:
        print(films.iloc[num]["id"],i[1],i[0])

In [ ]:
lst = calculate_distance(films, films.iloc[52822]["id"], films.iloc[52822]["prepDisc"])
lst = lst[:10]

for i in lst:
        print(films.iloc[52822]["id"],i[1],i[0])

In [ ]:
list(zip(list(films["prepDisc"]),list(films["id"])))